In [9]:
!pip install tensorflow torch torchvision
!pip install pytorch-pretrained-biggan


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [10]:
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
import torch
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from pytorch_pretrained_biggan import (
    BigGAN,
    one_hot_from_names,
    truncated_noise_sample,
    save_as_images,
    display_in_terminal
)

In [11]:
def load_melanoma_dataset(data_dir, batch_size=32, image_size=64):
    # Define the image transformations
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),  # Resize to 64x64
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)

    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    return dataloader

# Usage
data_dir = '/Users/andreshofmann/Desktop/Studies/Uol/7t/FP/stage_2/Images/img_patches/mel_patches'
dataset = load_melanoma_dataset(data_dir)

In [12]:
# data loading function
def load_melanoma_dataset(data_dir, batch_size=32, image_size=64):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    return dataloader

def fine_tune_biggan(model, dataset, num_epochs=100, learning_rate=1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()

    # Get the correct dimensions
    noise_dim = model.config.z_dim
    num_classes = model.config.num_classes

    # List to store average loss for each epoch
    losses = []

    for epoch in range(num_epochs):
        total_loss = 0
        for batch, _ in tqdm(dataset, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch = batch.to(device)
            batch_size = batch.size(0)

            # Generate random noise
            noise = torch.randn(batch_size, noise_dim, device=device)
            
            # Generate random class labels
            class_labels = torch.randint(0, num_classes, (batch_size,), device=device)
            
            # Convert class labels to one-hot encoding
            class_labels_one_hot = F.one_hot(class_labels, num_classes=num_classes).float()
            
            # Generate fake images
            fake_images = model(noise, class_labels_one_hot, truncation=0.4)
            
            # Resize fake images to match your dataset (64x64)
            fake_images = F.interpolate(fake_images, size=(64, 64), mode='bilinear', align_corners=False)
            
            # Compute loss
            loss = criterion(fake_images, batch)
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataset)
        losses.append(avg_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

    return model, losses

# Load your dataset
data_dir = '/Users/andreshofmann/Desktop/Studies/Uol/7t/FP/stage_2/Images/img_patches/mel_patches'  
dataset = load_melanoma_dataset(data_dir)

# Load pre-trained BigGAN
model = BigGAN.from_pretrained('biggan-deep-256')

# Fine-tune the model
fine_tuned_model, training_losses = fine_tune_biggan(model, dataset, num_epochs=100, learning_rate=1e-4)

# Save the fine-tuned model
torch.save(fine_tuned_model.state_dict(), 'fine_tuned_biggan.pth')
print("Fine-tuned model saved as 'fine_tuned_biggan.pth'")

# Plot the training losses
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(training_losses) + 1), training_losses)
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.grid(True)
plt.savefig('training_loss_plot.png')
plt.show()
print("Training loss plot saved as 'training_loss_plot.png'")

Epoch 1/100: 100%|██████████| 557/557 [12:01:19<00:00, 77.70s/it]   


Epoch 1/100, Average Loss: 0.1208


Epoch 2/100:  19%|█▊        | 104/557 [1:33:00<6:51:18, 54.48s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x35217a3e0>
Traceback (most recent call last):
  File "/Users/andreshofmann/.pyenv/versions/3.12.2/envs/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/Users/andreshofmann/.pyenv/versions/3.12.2/envs/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/andreshofmann/.pyenv/versions/3.12.2/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
from pytorch_pretrained_biggan import BigGAN
from tqdm import tqdm

def load_melanoma_dataset(data_dir, batch_size=32, image_size=64):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, persistent_workers=False)
    return dataloader

def fine_tune_biggan(model, dataset, num_epochs=100, learning_rate=1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()

    noise_dim = model.config.z_dim
    num_classes = model.config.num_classes

    losses = []

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(dataset, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, _ = batch  # Assuming the dataset returns (images, labels), and we only need images here
            images = images.to(device)
            batch_size = images.size(0)

            noise = torch.randn(batch_size, noise_dim, device=device)
            class_labels = torch.randint(0, num_classes, (batch_size,), device=device)
            class_labels_one_hot = F.one_hot(class_labels, num_classes=num_classes).float()

            fake_images = model(noise, class_labels_one_hot, truncation=0.4)
            fake_images = F.interpolate(fake_images, size=(64, 64), mode='bilinear', align_corners=False)

            loss = criterion(fake_images, images)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataset)
        losses.append(avg_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

    return model, losses

if __name__ == '__main__':
    import multiprocessing
    multiprocessing.set_start_method('spawn', force=True)

    data_dir = '/Users/andreshofmann/Desktop/Studies/Uol/7t/FP/stage_2/Images/img_patches/mel_patches'
    dataset = load_melanoma_dataset(data_dir)

    model = BigGAN.from_pretrained('biggan-deep-256')
    fine_tuned_model, training_losses = fine_tune_biggan(model, dataset, num_epochs=100, learning_rate=1e-4)

    torch.save(fine_tuned_model.state_dict(), 'fine_tuned_biggan.pth')
    print("Fine-tuned model saved as 'fine_tuned_biggan.pth'")

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(training_losses) + 1), training_losses)
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Average Loss')
    plt.grid(True)
    plt.savefig('training_loss_plot.png')
    plt.show()
    print("Training loss plot saved as 'training_loss_plot.png'")
